# House Prices 
<h4 style="color:red;">  use regression to Predict sales prices  </h4>

In [110]:
# import needed libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 

%matplotlib inline

##### read the data 

In [111]:
train_data = pd.read_csv('train.csv')
train_data.shape

(1460, 81)

# prepare the data 

In [113]:
X = train_data.drop('SalePrice' , axis=1)
y = train_data['SalePrice']


# make a function to clean the data to help me in the  test data
def clean(df) :

    # delete id column
    df.drop('Id' , inplace=True , axis=1)

    # delete columns that have null values more than 1000 
    to_null = df.isnull().sum()[df.isnull().sum() >1000].index.tolist()
    df.drop(to_null , inplace=True , axis=1)
    
    # get all string column
    string_cols = df.select_dtypes('object').columns
    # get the most frequent value in all string columns
    mode_values = df[string_cols].mode().iloc[0].values
    
    for col,ele in zip(string_cols , mode_values) :
    #fill the null with the mode
        df[col] = df[col].fillna(ele)
        df[col] = df[col].astype('category').cat.codes
  
    # replace the integer nan values with the mean
    df.fillna(df.mean() , inplace=True)
    
    # data scaing to z scores
    scaler = StandardScaler() 
    scaler.fit(df)
    X = scaler.transform(df)
    
    return df 

X = clean(X)

X.shape , y.shape

((1460, 75), (1460,))

# train the regression model

In [115]:

# split the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44, shuffle =True) 


# build the model
LinearRegressionModel = LinearRegression(fit_intercept=True , n_jobs=-1) 
LinearRegressionModel.fit(X_train, y_train) 

print('Linear Regression Train Score is : ' , LinearRegressionModel.score(X_train, y_train)) 
print('Linear Regression Test Score is : ' , LinearRegressionModel.score(X_test, y_test))
 

Linear Regression Train Score is :  0.8530279082833657
Linear Regression Test Score is :  0.8226290552387828


# read the test data file


In [129]:
test_data = pd.read_csv("test.csv")
test_data.shape

(1459, 80)

# make the submission file


In [127]:
submit = pd.DataFrame(columns = ['Id' , 'SalePrice'])
submit['Id'] = test_data['Id']
X_TEST = clean(test_data)
submit['SalePrice'] = LinearRegressionModel.predict(X_TEST)

submit.to_csv('submission.csv' , index=False)